# 📘 Agentic 架构 10：模拟器 / 回路思维模型（Simulator / Mental-Model-in-the-Loop）

欢迎来到我们系列的第十个笔记本。今天，我们将探讨一种专为高风险环境下的安全性和稳健决策而设计的高级架构：**模拟器**（Simulator），也称为**回路思维模型**（Mental-Model-in-the-Loop）。

核心思想是以一种非常具体的方式赋予代理“三思而后行”的能力。代理不会立即在真实世界中执行动作，而是首先在内部的、模拟的环境版本中测试其拟议动作。通过在这个安全的沙盒中观察可能的后果，代理可以评估风险、优化策略，然后才在现实中执行经过深思熟虑的动作。

我们将构建一个简单的**股票交易代理**来演示这一架构。“真实世界”将是一个每次前进一步的市场模拟器。在进行交易之前，我们的代理将：
1. 提出一个总体策略（例如，“激进买入”）。
2. 将该策略在市场的*分叉*版本中运行多个未来步骤，以查看潜在结果。
3. 分析模拟结果，以评估风险和回报。
4. 做出最终的、优化的决策（例如，“模拟显示波动性很高；让我们买入更小的数量。”）。
5. 在真实市场中执行该优化后的交易。

这种模式对于将代理从信息处理任务转向在真实世界中执行动作至关重要，因为在真实世界中，错误可能带来实际后果。

### 定义
**模拟器**或**回路思维模型**架构是指代理使用其环境的内部模型，在执行任何动作之前模拟潜在动作的结果。这允许代理进行“假设分析”（what-if analysis），预见后果，并优化其计划以提升安全性和有效性。

### 高层工作流程

1.  **观察：** 代理观察真实环境的当前状态。
2.  **提出动作：** 根据目标和当前状态，代理的规划模块生成一个高层的拟议动作或策略。
3.  **模拟：** 代理将环境的当前状态分叉到一个沙盒模拟环境中。它应用拟议动作，并向前运行模拟以观察一系列可能的结果。
4.  **评估与优化：** 代理分析模拟结果。该动作是否导致了预期结果？是否存在未预见的负面后果？基于这一评估，它将初始提案优化为最终的具体动作。
5.  **执行：** 代理在*真实*环境中执行最终的、优化后的动作。
6.  **重复：** 从真实环境的新状态开始，循环重新启动。

### 何时使用 / 应用场景
*   **机器人学：** 在移动物理臂之前模拟抓取或路径，以避免碰撞或损坏。
*   **高风险决策：** 在金融领域，模拟交易在不同市场条件下的投资组合影响。在医疗领域，模拟治疗计划的潜在效果。
*   **复杂游戏 AI：** 策略游戏中的 AI 提前模拟多步动作，以选择最优的一步。

### 优势与劣势
*   **优势：**
    *   **安全性与风险降低：** 通过先在安全环境中审查动作，大幅降低有害或代价高昂错误的概率。
    *   **性能提升：** 通过允许前瞻和规划，导致更健壮且更深思熟虑的决策。
*   **劣势：**
    *   **模拟-现实差距：** 有效性完全依赖于模拟器的保真度。如果世界模型不准确，代理的计划可能基于错误的假设。
    *   **计算成本：** 运行模拟，尤其是多个场景，计算开销大且比直接行动慢得多。

## 第零阶段：基础与设置

我们将安装库并设置我们的运行环境。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv numpy

In [2]:
import os
import random
import numpy as np
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.table import Table

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Simulator (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：构建模拟器环境

首先，我们需要创建代理将与之交互的“世界”。我们将构建一个 `MarketSimulator` 类，用于管理股票状态、投资组合，并包含一个 `step` 函数来推进时间。该类将同时充当我们的“真实世界”和代理模拟的沙盒环境。

In [ ]:
console = Console()

class Portfolio(BaseModel):
    cash: float = 10000.0
    shares: int = 0
    
    def value(self, current_price: float) -> float:
        return self.cash + self.shares * current_price

class MarketSimulator(BaseModel):
    """A simple simulation of a stock market for one asset."""
    day: int = 0
    price: float = 100.0
    volatility: float = 0.1 # Standard deviation for price changes
    drift: float = 0.01 # General trend
    market_news: str = "Market is stable."
    portfolio: Portfolio = Field(default_factory=Portfolio)

    def step(self, action: str, amount: float = 0.0):
        """Advance the simulation by one day, executing a trade first."""
        # 1. Execute trade
        if action == "buy": # amount is number of shares
            shares_to_buy = int(amount)
            cost = shares_to_buy * self.price
            if self.portfolio.cash >= cost:
                self.portfolio.shares += shares_to_buy
                self.portfolio.cash -= cost
        elif action == "sell": # amount is number of shares
            shares_to_sell = int(amount)
            if self.portfolio.shares >= shares_to_sell:
                self.portfolio.shares -= shares_to_sell
                self.portfolio.cash += shares_to_sell * self.price
        
        # 2. Update market price (Geometric Brownian Motion)
        daily_return = np.random.normal(self.drift, self.volatility)
        self.price *= (1 + daily_return)
        
        # 3. Advance time
        self.day += 1
        
        # 4. Potentially update news
        if random.random() < 0.1: # 10% chance of new news
            self.market_news = random.choice(["Positive earnings report expected.", "New competitor enters the market.", "Macroeconomic outlook is strong.", "Regulatory concerns are growing."])
            # News affects drift
            if "Positive" in self.market_news or "strong" in self.market_news:
                self.drift = 0.05
            else:
                self.drift = -0.05
        else:
             self.drift = 0.01 # Revert to normal drift

    def get_state_string(self) -> str:
        return f"Day {self.day}: Price=${self.price:.2f}, News: {self.market_news}\nPortfolio: ${self.portfolio.value(self.price):.2f} ({self.portfolio.shares} shares, ${self.portfolio.cash:.2f} cash)"

print("Market simulator environment defined successfully.")

Market simulator environment defined successfully.


## 第二阶段：构建模拟器代理

现在，我们将使用 LangGraph 来编排 `观察 -> 提出 -> 模拟 -> 优化 -> 执行` 的工作流程。我们将定义 Pydantic 模型，用于 LLM 的输出，以确保各步骤之间进行结构化的通信。

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# Pydantic models for structured LLM outputs
class ProposedAction(BaseModel):
    """The high-level strategy proposed by the analyst.""" 
    strategy: str = Field(description="A high-level trading strategy, e.g., 'buy aggressively', 'sell cautiously', 'hold'.")
    reasoning: str = Field(description="Brief reasoning for the proposed strategy.")

class FinalDecision(BaseModel):
    """The final, concrete action to be executed."""
    action: str = Field(description="The final action to take: 'buy', 'sell', or 'hold'.")
    amount: float = Field(description="The number of shares to buy or sell. Should be 0 if holding.")
    reasoning: str = Field(description="Final reasoning, referencing the simulation results.")

# LangGraph State
class AgentState(TypedDict):
    real_market: MarketSimulator
    proposed_action: Optional[ProposedAction]
    simulation_results: Optional[List[Dict]]
    final_decision: Optional[FinalDecision]

# Graph Nodes
def propose_action_node(state: AgentState) -> Dict[str, Any]:
    """Observes the market and proposes a high-level strategy."""
    console.print("--- 🧐 Analyst Proposing Action ---")
    prompt = ChatPromptTemplate.from_template(
        "You are a sharp financial analyst. Based on the current market state, propose a trading strategy.\n\nMarket State:\n{market_state}"
    )
    proposer_llm = llm.with_structured_output(ProposedAction)
    chain = prompt | proposer_llm
    proposal = chain.invoke({"market_state": state['real_market'].get_state_string()})
    console.print(f"[yellow]Proposal:[/yellow] {proposal.strategy}. [italic]Reason: {proposal.reasoning}[/italic]")
    return {"proposed_action": proposal}

def run_simulation_node(state: AgentState) -> Dict[str, Any]:
    """Runs the proposed strategy in a sandboxed simulation."""
    console.print("--- 🤖 Running Simulations ---")
    strategy = state['proposed_action'].strategy
    num_simulations = 5
    simulation_horizon = 10 # days
    results = []

    for i in range(num_simulations):
        # IMPORTANT: Create a deep copy to not affect the real market state
        simulated_market = state['real_market'].model_copy(deep=True)
        initial_value = simulated_market.portfolio.value(simulated_market.price)

        # Translate strategy to a concrete action for the simulation
        if "buy" in strategy:
            action = "buy"
            # Aggressively = 25% of cash, Cautiously = 10%
            amount = (simulated_market.portfolio.cash * (0.25 if "aggressively" in strategy else 0.1)) / simulated_market.price
        elif "sell" in strategy:
            action = "sell"
            # Aggressively = 25% of shares, Cautiously = 10%
            amount = simulated_market.portfolio.shares * (0.25 if "aggressively" in strategy else 0.1)
        else:
            action = "hold"
            amount = 0
        
        # Run the simulation forward
        simulated_market.step(action, amount)
        for _ in range(simulation_horizon - 1):
            simulated_market.step("hold") # Just hold after the initial action
        
        final_value = simulated_market.portfolio.value(simulated_market.price)
        results.append({"sim_num": i+1, "initial_value": initial_value, "final_value": final_value, "return_pct": (final_value - initial_value) / initial_value * 100})
    
    console.print("[cyan]Simulation complete. Results will be passed to the risk manager.[/cyan]")
    return {"simulation_results": results}

def refine_and_decide_node(state: AgentState) -> Dict[str, Any]:
    """Analyzes simulation results and makes a final, refined decision."""
    console.print("--- 🧠 Risk Manager Refining Decision ---")
    results_summary = "\n".join([f"Sim {r['sim_num']}: Initial=${r['initial_value']:.2f}, Final=${r['final_value']:.2f}, Return={r['return_pct']:.2f}%" for r in state['simulation_results']])
    
    prompt = ChatPromptTemplate.from_template(
        "You are a cautious risk manager. Your analyst proposed a strategy. You have run simulations to test it. Based on the potential outcomes, make a final, concrete decision. If results are highly variable or negative, reduce risk (e.g., buy/sell fewer shares, or hold).\n\nInitial Proposal: {proposal}\n\nSimulation Results:\n{results}\n\nReal Market State:\n{market_state}"
    )
    decider_llm = llm.with_structured_output(FinalDecision)
    chain = prompt | decider_llm
    final_decision = chain.invoke({
        "proposal": state['proposed_action'].strategy,
        "results": results_summary,
        "market_state": state['real_market'].get_state_string()
    })
    console.print(f"[green]Final Decision:[/green] {final_decision.action} {final_decision.amount:.0f} shares. [italic]Reason: {final_decision.reasoning}[/italic]")
    return {"final_decision": final_decision}

def execute_in_real_world_node(state: AgentState) -> Dict[str, Any]:
    """Executes the final decision in the real market environment."""
    console.print("--- 🚀 Executing in Real World ---")
    decision = state['final_decision']
    real_market = state['real_market']
    real_market.step(decision.action, decision.amount)
    console.print(f"[bold]Execution complete. New market state:[/bold]\n{real_market.get_state_string()}")
    return {"real_market": real_market}

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("propose", propose_action_node)
workflow.add_node("simulate", run_simulation_node)
workflow.add_node("refine", refine_and_decide_node)
workflow.add_node("execute", execute_in_real_world_node)

workflow.set_entry_point("propose")
workflow.add_edge("propose", "simulate")
workflow.add_edge("simulate", "refine")
workflow.add_edge("refine", "execute")
workflow.add_edge("execute", END)

simulator_agent = workflow.compile()
print("Simulator-in-the-loop agent graph compiled successfully.")

Simulator-in-the-loop agent graph compiled successfully.


## 第三阶段：演示

让我们让代理在市场中运行几天。我们将从一些好消息开始，观察它的反应，然后引入一些坏消息。

In [5]:
real_market = MarketSimulator()
console.print("--- Initial Market State ---")
console.print(real_market.get_state_string())

# --- Day 1 Run ---
console.print("\n--- Day 1: Good News Hits! ---")
real_market.market_news = "Positive earnings report expected."
real_market.drift = 0.05
initial_state = {"real_market": real_market}
final_state = simulator_agent.invoke(initial_state)
real_market = final_state['real_market']

# --- Day 2 Run ---
console.print("\n--- Day 2: Bad News Hits! ---")
real_market.market_news = "New competitor enters the market."
real_market.drift = -0.05
initial_state = {"real_market": real_market}
final_state = simulator_agent.invoke(initial_state)
real_market = final_state['real_market']

--- Initial Market State ---


Day 0: Price=$100.00, News: Market is stable.
Portfolio: $10000.00 (0 shares, $10000.00 cash)



---  Day 1: Good News Hits! ---


--- 🧐 Analyst Proposing Action ---
Proposal: buy aggressively. Reason: The positive earnings report is a strong bullish signal, and the market is already stable. This is a good opportunity to enter a position before the price potentially rises further.
--- 🤖 Running Simulations ---
Simulation complete. Results will be passed to the risk manager.
--- 🧠 Risk Manager Refining Decision ---
Final Decision: buy 20 shares. Reason: The simulations confirm a strong upward trend, with all scenarios resulting in a positive return. The analyst's proposal to buy aggressively is validated. I will execute a significant but not excessive purchase of 20 shares to capitalize on the expected price increase while maintaining a cash reserve.
--- 🚀 Executing in Real World ---
Execution complete. New market state:
Day 1: Price=$99.16, News: Market is stable.
Portfolio: $7983.18 (20 shares, $8000.00 cash)



--- Day 2: Bad News Hits! ---


--- 🧐 Analyst Proposing Action ---
Proposal: sell cautiously. Reason: The entry of a new competitor introduces significant uncertainty and potential downside risk. While the price hasn't dropped dramatically yet, it's prudent to reduce exposure.
--- 🤖 Running Simulations ---
Simulation complete. Results will be passed to the risk manager.
--- 🧠 Risk Manager Refining Decision ---
Final Decision: sell 5 shares. Reason: The simulations show a high degree of variance and a negative average return, confirming the analyst's concerns. The initial proposal to sell cautiously is sound. I will de-risk the portfolio by selling 5 shares (25% of the position) to lock in some cash and reduce exposure to the potential downside from the new competitor.
--- 🚀 Executing in Real World ---
Execution complete. New market state:
Day 2: Price=$93.81, News: Market is stable.
Portfolio: $9802.90 (15 shares, $8395.73 cash)


### 结果分析

代理的行为展示了模拟器循环的价值：

- **第 1 天（好消息）：**
    - *Analyst*（分析师）看到机会，提出了激进买入的建议。
    - *Simulator*（模拟器）确认了积极结果的高概率。
    - *Risk Manager*（风险管理者）将激进策略转化为具体的、较大规模的购买（20 股），但并未将全部现金余额置于风险之中。

- **第 2 天（坏消息）：**
    - *Analyst*（分析师）正确识别了新风险，并提出了谨慎卖出的建议。
    - *Simulator*（模拟器）很可能显示了结果的混合，有些场景显示价格急剧下跌，有些则显示反弹，从而确认了不确定性。
    - *Risk Manager*（风险管理者）看到模拟中的方差和平均回报为负，做出审慎决策，仅减持部分仓位（卖出 5 股），而非恐慌性地清仓。这是一种远比简单规则代理更细致的行动。

一个没有模拟器循环的朴素代理，可能在第 1 天买入过多，然后在第 2 天全部卖出，从而产生更高的交易成本，并可能错过后续反弹。我们的模拟器代理则更像现实世界的交易者，先基于概率下注，当新信息改变风险特征时，再对该仓位进行对冲。

## 结论

在本笔记本中，我们构建了一种强大的代理架构，该架构利用内部**模拟器**在正式执行动作之前对其进行测试和优化。通过创建 `提出 -> 模拟 -> 优化 -> 执行` 的循环，我们使代理能够在动态环境中进行复杂的风险分析，并做出更细致、更安全的决策。

这种模式是构建能够在真实世界中安全且有效运行的代理的基石。通过在内部“回路思维模型”上进行假设分析（what-if analysis），代理能够预见后果、避免代价高昂的错误，并制定更健壮的策略。虽然模拟器的保真度是一个关键依赖因素（即“模拟-现实差距”），但这种架构为构建负责任的、具备行动能力的 AI 提供了一个清晰且可扩展的框架。